---
title: "Supplementary materials"
format:
  pdf:
    toc: true
    fig-pos: 'H'
    tbl-pos: 'H'
    fig-cap-location: bottom
    tbl-cap-location: top
    include-in-header:
      text: |
        \renewcommand{\familydefault}{\rmdefault}
        \usepackage{sectsty}
        \allsectionsfont{\rmfamily}
        \usepackage[labelfont=bf]{caption}
        \usepackage{float}
execute:
  echo: false
  warning: false
jupyter: python3
bibliography: ../supplementary/tb_refs.bib
csl: https://www.zotero.org/styles/the-lancet
---

In [ ]:
from IPython.display import Image, display, Markdown
from tb_incubator.constants import set_project_base_path, image_path, ImplementCDR
from tb_incubator.calibrate import get_all_priors, tabulate_calib_results, tabulate_priors
from tb_incubator.utils import load_full_param_info, create_parameter_table
from tb_incubator.input import load_targets
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
import arviz as az
import numpy as np
pio.renderers.default = 'svg'


project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]
out_path = project_paths["OUTPUTS"]

In [ ]:
params_full = load_full_param_info()
file_suffix = "cdr_notif_25000d10000t_01nr"
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')
burnt_idata = idata.sel(draw=np.s_[2500:])
targets = load_targets()

# Model structure 

## Compartments

We constructed our TB dynamics model using `summer` framework [@shipmanSummerepi2]. The base model consists of six compartments that represent key states for TB infection and disease in a population: 

- Susceptible (S): Individuals who have never had TB and are at risk of infection with Mycobacterium tuberculosis (M. tb).
- Early latent (E): Individuals who have recently contracted M. tb but have not yet developed active TB symptoms. During the early period following initial infection, they face an elevated risk of progressing to active TB.
- Late latent (L): Individuals who have had a longer period of latent TB infection compared to those in the early latent compartment. The likelihood of progressing to active TB is lower than in early latent infection due to immune containment of the pathogen. Individuals in this compartment may be reinfected and reenter the 'E' compartment.
- Infectious (I): Individuals with active TB disease who can transmit the infection to susceptible individuals.
- Treatment (T): Individuals who are receiving TB treatment.
- Recovered (R): Individuals who have recovered from active TB disease through self-recovery or treatment completion. Individuals in this compartment may be reinfected and move back to the 'E' compartment.



## Demographics

Our model runs from 1800 to 2035. Initially, we introduced one susceptible person as the starting population. As the population grows, we seed infectious individuals into the population to allow *M. tuberculosis* infection to emerge and spread naturally throughout the simulated period.

Population estimates from the United Nations’ *World Population Prospects* [@unwpp2024] were used to inform the model. We assumed that each death in the population is replaced by a birth. To represent population growth, we used a time-varying population entry rate function, calculated based on annual population changes and the duration of the run-in period. The detailed steps and equations are as follows:

1. Calculate annual total population [@unwpp2024] and the duration of the run-in period:

   $$
   P(t) = \sum_{i=1}^{n} \text{Population}_i(t)
   $$

   $$
   T_{\text{start}} = t_0 - t_{\text{model}}
   $$

2. Calculate population change for years after the initial data year ($t > t_0$):

   $$
   \Delta P(t) = P(t) - P(t-1)
   $$

3. Calculate population change for the initial data year ($t = t_0$):

   $$
   \Delta P(t_0) = \frac{P(t_0)}{T_{\text{start}}}
   $$

4. Define the population entry rate function:

   $$
   \lambda(t) = f_{\text{sigmoid}}\left(\{(t_i, \Delta P(t_i))\}_{i=1}^{m}\right)
   $$

Here, $P(t)$ denotes the total population at time $t$, $\text{Population}_i(t)$ represents the population of group $i$ at time $t$, and $n$ is the number of population groups. The duration of the run-in period ($T_{\text{start}}$) is defined using the first year of available population data ($t_0$, e.g. 1950) and the model start time ($t_{\text{model}}$, e.g. 1800). The net population change ($\Delta P(t)$) depends on the period of interest (initial or later years). The population entry rate ($\lambda(t)$) is derived by interpolating the ordered set of time points and their corresponding population changes, $\{(t_i, \Delta P(t_i))\}$, using a sigmoidal function. The total number of time points is denoted by $m$.

We assumed that the initial population $P(t_0)$ accumulated uniformly over the run-in period and that age stratification stabilised by the start of the analysis. Time-varying non-TB-related mortality was applied across all compartments to account for deaths from non-TB causes, which were later stratified by age.


## *M. tb* transmission

### Natural history of TB

The rates of TB-related mortality and self-recovery were modelled based on estimates from a previous study [@ragonnet2020]. These rates vary depending on the organ status (i.e., smear-positive, smear-negative, and extrapulmonary TB), with assumption that extrapulmonary TB follows the same rates as smear-negative TB.

### Latency flows
We used estimates from previous study to inform the dynamics of progression from latent to active TB [@ragonnet2017]. These parameters vary with the age and a multiplier was incorporated to accommodate for the uncertainties around these progression rates as previously done in [@ragonnet2017].

### Passive detection of individuals with active TB disease
The detection or treatment commencement rate represents the progression of individuals with active TB (I) to the treatment (T) compartment, based on the assumption that all diagnosed individuals receive immediate treatment upon detection. We modelled the passive screening rate as a time-varying function, under the assumption that screening effectiveness would continuously improve over time, similar to the previous study [@ragonnet2022]. The mathematical representation of this function is as follows:

$$
t → 0.5 (tanh⁡(m(t-c)+1)) \cdot final
$$

We define $c$ as the time at which the curve inflects, $m$ as the gradient value at the inflection point, and $final$ as the value of upper asymptote. The value of these parameters was calibrated to fit the target data. 

We extended this function to incorporate improvements in TB diagnostic capacity over time, primarily through increased deployment and utilisation of Xpert testing [@fig-xpertutil]. We also accounted for COVID-related detection reduction and the subsequent rebound of case notifications, to capture the impact of the COVID-19 pandemic which heavily affected the national healthcare system. This disruption was implemented by applying a time-variant multiplier during the pandemic on the treatment commencement rate.

The diagnostic capacity, $D(t)$, represents the overall probability that an individual with active TB will be detected by the diagnostic system at time $t$. This was modeled as:
$$
D(t) = D_0 + (1 - D_0) \cdot \sigma_X \cdot U_X(t)
$$
where $D_0$ represents the baseline diagnostic capacity using conventional diagnostic methods (e.g., smear microscopy, clinical diagnosis) prior to Xpert implementation, $\sigma_X$ is the sensitivity of the Xpert assay, and $U_X(t)$ is the time-varying Xpert capacity utilisation rate at time $t$, derived from the annual number of Xpert tests performed divided by the maximum annual testing capacity of available Xpert machines [@moh2024report].

The term $(1 - D_0)$ represents the diagnostic gap, which is the proportion of TB cases that would remain undetected using only conventional methods. The product of $(1 - D_0) \cdot \sigma_X \cdot U_X(t)$ captures the incremental detection capacity gained through Xpert testing, which depends on both the Xpert sensitivity and the extent to which available Xpert testing capacity is utilised over time. 

## Treatment outcomes

We modelled three different outcomes of treatment: cured, relapse, and death. The proportion of these outcomes are calculated as follows:
$$
P_{\text{cure}} = S
$$
$$
P_{\text{TB death}} = (1-S) \times \Pi
$$
$$
P_{\text{natural death}} = 1 - e^{-\mu T}
$$
$$
P_{\text{relapse}} = 1 - P_{\text{cure}} - P_{\text{TB death}} - P_{\text{natural death}}
$$
Here, $P$ denotes proportion, $S$ is the treatment success proportion (often referred to as 'treatment success rate'), representing the cure rate. $\Pi$ represents the fraction of unsuccessful treatment outcomes resulting in death, $T$ is the treatment period, and $\mu$ is the natural death rate. Finally, the transmission rate for each outcome is obtained by multiplying the proportion of the corresponding outcome by the treatment period ($T$). These values are then adjusted by age since non-TB-related death varies by age.

In [ ]:
#| label: fig-xpertutil
#| fig-cap: "Utilisation of Xpert assay for TB diagnosis in 2016-2023 [@moh2024report]"
fig = go.Figure(data=go.Scatter(
    x=targets['genexpert_utilisation'].index[1:], 
    y=targets['genexpert_utilisation'].iloc[1:]*100, 
    mode='markers+lines',
    marker=dict(size=10, color='blue', symbol='circle')
))

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Xpert utilisation (%)",
    template="plotly_white",
    font_family="Space Grotesk",
    title_font_family="Space Grotesk",
    margin=dict(l=10, r=10, t=10, b=10),
    width=500,
    height=300
)

pio.show(fig)

## Stratification

### Stratification by age
We stratified all compartments of the model based on 5 age groups: 0-4, 5-14, 15-34, 35-49, and 50+ years, with assumption that interactions between and across age group are homogeneous. Age group-specific adjustments were applied for population death flows, latency flows, infectiousness, and treatment outcomes.

### Stratification by clinical form of TB
To capture clinical manifestation variations in TB, we stratified individuals based on their disease form, which involves different organ and smear status: smear-positive pulmonary TB, smear-negative pulmonary TB, and extrapulmonary TB. This stratification was applied to compartments that represents individuals with active TB disease, such as 'infectious' and 'treatment' compartments, with consideration that each form of TB varies in terms of disease severity, infectiousness, and detection rate.

# Model parameterisation and calibration

## Parameterisation
The parameters used in this model represent population characteristics, infection dynamics of *Mycobacterium tuberculosis* (*M. tb*), the natural history of TB, and TB control measures. Detailed information on the parameters is shown in @tbl-parameters. Estimates for infection dynamics and natural history parameters were obtained from previous studies, while the remaining parameters were either derived from other studies or adjusted as needed. We further calibrate our model to fit the local data on Indonesia’s TB case notifications [@whotb2023] and prevalence for bacteriologically confirmed pulmonary TB per 100,000 population in people aged over 15 years, based on the 2013-2014 national TB Prevalence survey [@moh2015] @tbl-targets. 

In [ ]:
#| tbl-colwidths: [30,25,40]
#| tbl-cap: "Model parameters."
#| label: tbl-parameters
priors = get_all_priors(
    covid_effects=True, 
    apply_diagnostic_capacity=True, 
    apply_cdr=ImplementCDR.ON_NOTIFICATION, 
    xpert_improvement=True
)
fixed_params_info = load_full_param_info()

df = create_parameter_table(
    priors,
    fixed_params_info,
    additional_params=[
        {
            "Parameter": "Universal death rate",
            "Value": "Time-variant",
            "Unit": "",
            "Source": "UN World Population Division[@unwpp2024]",
            "Group": "Demographics"
        }
    ]
)
df = pd.DataFrame(df)
df.to_csv(out_path / 'results_table_params.csv', index=True)
Markdown(df.to_markdown(index=False))

## Calibration

The model was calibrated using Bayesian inference with the differential evolution Metropolis-Z (DEMetropolisZ) algorithm implemented in PyMC [@salvatier2016]. This sampler uses past samples from multiple parallel chains to inform new proposals, improving parameter space exploration and convergence reliability.

We first generated 16 parameter sets using Latin hypercube sampling with 67% confidence interval [@dutta2020] to ensure broad coverage of initial distributions of the multidimensional parameter space. These samples were ranked by log-likelihood and then optimised in parallel using Nevergrad's TwoPointsDE algorithm [@nevergrad2018], with 500 function evaluations allocated to each optimisation. The eight parameter sets with the highest post-optimization log-likelihood were selected as starting points for eight parallel Markov chains in the DEMetropolisZ sampler. This proceeded for 10,000 tuning cycles and 25,000 sampling cycles. After calibration, we generated 95% credible intervals by running the model with 2,000 parameter sets from accepted samples of the calibration phase. Each model candidate completed 8 chains of 35,000 iterations per chain within 8 hours on a computer with an 8-core Apple M1 processor and 16 GB of RAM. The best model was identified by comparing the expected log pointwise predictive density Watanabe-Akaike Information Criterion (EPLD-WAIC) values across different candidates, with higher values reflecting better predictive accuracy.

In [ ]:
#| tbl-colwidths: [35,20,20,30,30]
#| tbl-cap: "Calibration targets."
#| label: tbl-targets
data = {
    'Target': ['TB case notifications'] * 10 + ['Adult pulmonary TB prevalence (per 100,000)'],
    'Year': list(range(2014, 2024)) + [2014],
    'Value': [
        '322,806', '331,703', '364,671', '442,172', '568,865',
        '559,847', '384,025', '432,577', '708,658', '804,836',
        '759.1'
    ],
    'Uncertainty': ['—'] * 10 + ['589.7–960.8'],
    'Source': ['WHO [@who2024-tbrep]'] * 10 + ['Prevalence Survey 2013–2014 [@moh2015]']
}
df = pd.DataFrame(data)
df.columns = [f"**{col}**" for col in df.columns]

Markdown(df.to_markdown(index=False))

In [ ]:
#| tbl-cap: "Prior distributions for calibrated parameters."
#| label: tbl-priors
priors_table = tabulate_priors(priors, params_full)
priors_table = pd.DataFrame(priors_table)
priors_table.columns = [f"**{col}**" for col in priors_table.columns]
priors_table.to_csv(out_path / 'results_table_priors.csv', index=True)
Markdown(priors_table.to_markdown(index=True))

# Additional results

In [ ]:
#| label: fig-hist
#| fig-cap: "Total population and TB epidemic trajectory under the best-fitting candidate model configuration. Green solid lines represent the median of model estimates and shaded areas represent the interquartile ranges (dark shades) and 95% CI (light shades). Black filled points represent the empirical target data or estimates on TB epidemiology."
display(Image(filename=image_path/ 'hist.png'))

In [ ]:
#| label: fig-post
#| fig-cap: "Comparison of the model's prior and posterior parameter distributions under CDR on notification assumption. Grey shaded areas represent the prior distributions, while blue shaded areas represent the posterior distributions."
display(Image(filename=image_path/ 'pp_comp.png'))

In [ ]:
#| label: fig-xpert
#| fig-cap: "Projected effect of increased Xpert utilisation by 2027. Black solid lines represent the baseline (status quo) scenario. Each coloured solid line represents different Xpert utilisation, ranging from 70% to 95%. Shaded areas show interquartile ranges (dark) and 95% credible intervals (light). Black point indicates the national TB elimination targets for TB incidence while red points indicate the End TB targets set by the WHO. "
display(Image(filename=image_path/ 'scenario_xpert.png'))

In [ ]:
#| label: fig-detect
#| fig-cap: "Projected effect of increased passive case detection by 2027. Black solid lines represent the baseline (status quo) scenario. Each coloured solid line represents a different fold increase, ranging from 2- to 5-fold. Shaded areas show interquartile ranges (dark) and 95% credible intervals (light). Black point indicates the national TB elimination targets for TB incidence while red points indicate the End TB targets set by the WHO. "
display(Image(filename=image_path/ 'scenario_detection.png'))

In [ ]:
#| label: fig-tsr
#| fig-cap: "Projected effect of increased treatment success rate (TSR)  by 2027. Black solid lines represent the baseline (status quo) scenario. The light blue solid line represents TSR of 90% and the blue line represents TSR of 95%. Shaded areas show interquartile ranges (dark) and 95% credible intervals (light). Black point indicates the national TB elimination targets for TB incidence while red points indicate the End TB targets set by the WHO. "
display(Image(filename=image_path/ 'scenario_tsr.png'))

In [ ]:
#| label: fig-acf
#| fig-cap: "Projected effect of increased ACF coverage. Black solid lines represent the baseline (status quo) scenario. Each coloured solid line represents a different fold increase, ranging from 2- to 5-fold. Shaded areas show interquartile ranges (dark) and 95% credible intervals (light). Black point indicates the national TB elimination targets for TB incidence while red points indicate the End TB targets set by the WHO. "
display(Image(filename=image_path/ 'scenario_acf.png'))

In [ ]:
#| label: fig-comb
#| fig-cap: " Projected effect of combination scenarios: Scenario 1 and Scenario 2.  Scenario 1 is a combination of increasing Xpert utilisation to 70%, treatment success rate (TSR) to 90%, and 2-fold treatment initiation rate by 2027, and annual ACF implementation in 1% of population. Scenario 2 is a combination of increasing Xpert utilisation to 95%, 95% TSR, and 5-fold treatment initiation rate by 2027, and annual ACF implementation in 60% of population . Coloured solid lines represent different intervention combinations as described in the legend. Shaded areas show interquartile ranges (dark) and 95% credible intervals (light). Black points indicate the national TB elimination  targets for TB incidence and red points represent the 2035 End TB targets for incidence and mortality. "
display(Image(filename=image_path/ 'scenario_prog_comb_supp.png'))

In [ ]:
#| tbl-cap: "Model comparison using Expected Log Pointwise Predictive Density (ELPD) - Watanabe-Akaike Information Criterion. Predictive Density (ELPD) values with differences from the best model (ELPD Diff = 0). Higher ELPD indicates better predictive performance. CDR = case detection rate; p = effective number of parameters."
#| label: tbl-waic

df = pd.read_csv(out_path / 'results_table_waic.csv')
df['Model'] = ["No CDR", "CDR on notifications", "CDR on treatment commencement"] 
df = df.rename(columns={'elpd_waic': 'ELPD WAIC', 'p_waic': 'p WAIC', 'se': 'SE'})
df = df.drop(['weight', 'elpd_diff', 'dse', 'warning', 'scale', 'Unnamed: 0'], axis=1)
df.insert(0, df.columns[-1], df.pop(df.columns[-1]))
Markdown(df.to_markdown(index=False))

In [ ]:
#| tbl-cap: "Calibration metrics."
#| label: tbl-calib
results_table = tabulate_calib_results(burnt_idata, params_full)
#results_table.to_csv(out_path / 'results_table_calib.csv', index=True)
results_table.columns = [f"**{col}**" for col in results_table.columns]
Markdown(results_table.to_markdown())

# References